In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from dask.dot import dot_graph
from bundle import FPGA, fpga_op
from bundle import fpgapy as fp
from pyclk import Trace

%matplotlib inline

In [ ]:
iter_nb = 2
add_nb = 3
mul_nb = 3
func_nb = add_nb + mul_nb

fpga = FPGA(iter_nb, add_nb, mul_nb) #, cycle_nb = 1000)

trace = Trace()
trace.add(fpga.u_crossbar.r_iter_i)
trace.add(fpga.u_crossbar.r_func_i)
for i in range(iter_nb):
    trace.add(fpga.u_iterator[i].o_op_done)
    trace.add(fpga.u_crossbar.r_iter_func_i[i])
    trace.add(fpga.u_crossbar.r_iter_busy[i])
for i in range(func_nb):
    trace.add(fpga.u_func[i].o_res)
    trace.add(fpga.u_crossbar.r_func_busy[i])

fpga.set_trace(trace)

In [ ]:
size = 1000

a = fp.arange(0, size)
b = fp.ones(size)
c = a + b
d = c * a

a_true = np.arange(0, size)
b_true = np.ones(size)
c_true = a_true + b_true
d_true = c_true * a_true

In [ ]:
#dot_graph(d.dask)

In [ ]:
d = d.compute()
if np.array_equal(d, d_true):
    print('Success!')
else:
    print('Failed!')

In [ ]:
trace.plot()

In [ ]:
diff_idx = np.where(d != d_true)
plt.figure(figsize=(15, 5))
plt.scatter(np.arange(d_true.size), d_true, c='b')
plt.scatter(diff_idx, d[diff_idx], c='r', label='FPGA')
plt.scatter(diff_idx, d_true[diff_idx], c='g', label='NumPy')
#plt.scatter(diff_idx, z_true[diff_idx]-z[diff_idx], c='r')
plt.xlim(0, size)
plt.ylim(1, size+1)
plt.legend(loc='upper left')
plt.title('Differences')
plt.show()

In [ ]:
d[diff_idx]

In [ ]:
d_true[diff_idx]